In [11]:
import pandas as pd
import pyodbc
import requests
import os
import base64
from datetime import datetime
from openpyxl import Workbook


# SQL Server connection details
server = 'xxx.xxx.xxx.xx'
database = 'xxxxxxx'
username = 'xxxxxx'
password = 'xxxxx'



client_id = 'xxxxxxxx-xx-xxx-xxxx-xxxxxxxxx'  
client_secret = 'xxxx~xxxxxxx-xxxxxxx'  
tenant_id = 'xxxxxx-xxxx-xxx-xxx-xxxxx'  
  

# Mapping of persons to email IDs (ensure each person is mapped uniquely)
email_mapping = {
    "xxx xxxx": "ashish.ahirwar@xxxx.com",
    # "Nirali Shah": "ajay.singh@xxxxxxx.com",
    "xxxx Sinha": "ashish.xxx@xxxx.com"  # Add more mappings
}

# Function to get access token from Microsoft Graph API
def get_access_token():
    url = f'https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token'
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    data = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
        'scope': 'https://graph.microsoft.com/.default'
    }
    
    response = requests.post(url, headers=headers, data=data)
    return response.json().get("access_token")



def send_email_via_graph(recipient_email, attachment_path):
    access_token = get_access_token()
    
    # Get current date for the subject
    report_date = datetime.today().strftime("%d-%m-%Y")
    subject = f"RFM Automated Report ({report_date})"
    
    # Tech-Axion logo (Secure HTTPS URL)
    techaxion_logo_url = "https://www.xxxxx.com/wp-content/uploads/2022/10/Axion-Logo.png"
    cc_recipients = ["ashish.ahirwar@xxxxx.com"]

    # Email body with explanation & logo
    email_body = f"""
    <p>Kindly find the attached <b>RFM (Recency, Frequency, Monetary) automated reports</b> ({report_date}).</p>
    <p>Inside the report, you will find Clients bifurcated by the Recency, Frequency & Monetary model with (1,2,3) based ranking.</p>
    
    <h3>Matrix:</h3>
    <ul>
        <li>Matrix value <b>111</b> represents the lowest value (least recent, least frequent, lowest monetary value).</li>
        <li>Matrix value <b>333</b> represents the highest value (most recent, most frequent, highest monetary value).</li>
        <li>Matrix value <b>113</b> represents the least recent, least frequent, but higher monetary value. Similar logic applies to other Matrix values.</li>
    </ul>

    <h3>Category:</h3>
    <p>Taking reference from Matrix values, Clients are divided into categories like <b>Champions, Loyal Customers, Potential Loyalists, Promising, New Customers, At Risk, Can't Lose Them, Hibernating</b>.</p>

    <h3>Description, Characteristics, Actions:</h3>
    <p>This section provides insights into clients divided into different categories and the recommended actions to be taken accordingly.</p>

    <p>Thanks & Regards,</p>
    <p><b>Ashish Ahirwar</b><br>
    BI Developer (Data Analyst)<br>
    Tech-Axion</p>

    <img src="{techaxion_logo_url}" alt="TechAxion Logo" width="200">
    """

    # Read the attachment and encode it in Base64
    with open(attachment_path, "rb") as attachment_file:
        attachment_base64 = base64.b64encode(attachment_file.read()).decode('utf-8')

    # Email payload with attachment
    email_payload = {
        "message": {
            "subject": subject,
            "body": {
                "contentType": "HTML",
                "content": email_body
            },
            "toRecipients": [
                {"emailAddress": {"address": recipient_email}}
            ],
            "ccRecipients": [  # Adding CC recipients
                {"emailAddress": {"address": cc}} for cc in cc_recipients
            ],
            "attachments": [
                {
                    "@odata.type": "#microsoft.graph.fileAttachment",
                    "name": os.path.basename(attachment_path).split("/")[-1],  # Extracts the file name
                    "contentBytes": attachment_base64
                }
            ]
        }
    }

    # Send the email
    url = f"https://graph.microsoft.com/v1.0/users/{recipient_email}/sendMail"
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }

    response = requests.post(url, headers=headers, json=email_payload)

    # Check if the email was sent successfully
    if response.status_code == 202:
        print(f"Email sent successfully to {recipient_email}")
    else:
        print(f"Failed to send email to {recipient_email}: {response.text}")


# Connect to SQL Server and retrieve data
conn = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}')
query = """
            SELECT FY, CLIENT_ID, CLIENT_NAME, AGREEMENT_DATE, FIRST_TRADE, LAST_TRADE, FIRST_TRADE_DIFF, INACTIVE_SINCE, NO_OF_TRADES, TRADED_DAYS, 
            MKT_SESSIONS, GROSS_BRK, NET_BRK, TURNOVER, TOTAL_HOLDINGS, TOTAL_AUM, BRANCH_CODE, BRANCH_NAME, LEVEL_1, LEVEL_2, LEVEL_3, LEVEL_4,
            LEVEL_5, DIRECT_SUB_BROKER, INACTIVE_BUCKET, CATEGORY,DESCRIPTION, CHARACTERISTICS, ACTIONS, FILE_DATE, EXECUTION_TIME
            FROM RFM_MATRIX_PERC_SP 
            WHERE FY = 'FY 24-25' 
            """  # Replace 'your_table' with actual table name
df = pd.read_sql(query, conn)
conn.close()

# Ensure level columns exist
level_columns = ["LEVEL_1", "LEVEL_2", "LEVEL_3", "LEVEL_4"]
for col in level_columns:
    if col not in df.columns:
        print(f"Column {col} not found in data!")
        exit()

# Track processed persons to avoid duplicate reports
processed_persons = set()

# Iterate through level_1 to level_4
for level in level_columns:
    for person in df[level].dropna().unique():
        if person in processed_persons:
            continue  # Skip if already processed in a higher level
        
        # Filter data for this person
        person_data = df[df[level] == person]
        
        # Check if this person has an assigned email
        if person not in email_mapping:
            print(f"Skipping {person} (No email assigned)")
            continue
        
        recipient_email = email_mapping[person]
        
        # Generate Excel file
        timestamp = datetime.now().strftime("%Y%m%d")
        filename = f"{person.replace(' ', '_')}_Report_{timestamp}.xlsx"
        filepath = os.path.join(os.getcwd(), filename)
        
        with pd.ExcelWriter(filepath, engine="openpyxl") as writer:
            person_data.to_excel(writer, index=False, sheet_name="Report")
        
        print(f"Generated file: {filename}")
        
        # Send email
        email_subject = f"Report for {person}"
        email_body = f"Dear {person},\n\nPlease find your attached report.\n\nBest regards."
        send_email_via_graph(recipient_email, filepath)
        
        # Mark this person as processed
        processed_persons.add(person)

print("Process completed.")


C:\Users\ashish.ahirwar\AppData\Local\Temp\ipykernel_16984\3118636523.py:256: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Generated file: Praveen_Chaudhary_Report_20250227.xlsx
Email sent successfully to ashish.ahirwar@techaxion.com
Skipping HO Mgt (No email assigned)
Skipping Nirali Shah (No email assigned)
Skipping Himayat Ali (No email assigned)
Generated file: Amarendra_Sinha_Report_20250227.xlsx
Email sent successfully to ashish.ahirwar@techaxion.com
Skipping Vikas Pandya (No email assigned)
Skipping PCG Desk (No email assigned)
Skipping Mayur Soni (No email assigned)
Skipping Sub Broker (No email assigned)
Skipping Vijay Virkar (No email assigned)
Skipping Bhanu Prakash Mishra (No email assigned)
Skipping Sunil Deshpande (No email assigned)
Skipping Smita Mane (No email assigned)
Skipping Bhavesh Jobanputra (No email assigned)
Skipping Inactive (No email assigned)
Skipping Ravi Srivastava (No email assigned)
Skipping Trades App (No email assigned)
Skipping Nirali Shah (No email assigned)
Skipping Dinesh Modi (No email assigned)
Skipping Vinod P (No email assigned)
Skipping Dealing HO (No email assig

In [39]:
import pandas as pd
import pyodbc
import requests
import os
import base64
from datetime import datetime
from openpyxl import Workbook, load_workbook


# SQL Server connection details
server = 'xxx.xxx.xxx.xx'
database = 'xxxxxxx'
username = 'xxxxxx'
password = 'xxxxx'



client_id = 'xxxxxxxx-xx-xxx-xxxx-xxxxxxxxx'  
client_secret = 'xxxx~xxxxxxx-xxxxxxx'  
tenant_id = 'xxxxxx-xxxx-xxx-xxx-xxxxx' 

# Mapping of persons to email IDs (ensure each person is mapped uniquely)
email_mapping = {
    "Praveen Chaudhary": "ashish.xxxx@xxxx.com",
    # "Nirali Shah": "ajay.singh@xxx.com",
    "Amarendra Sinha": "ashish.xxx@xxxxx.com"  # Add more mappings
}

# Function to get access token from Microsoft Graph API
def get_access_token():
    url = f'https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token'
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    data = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
        'scope': 'https://graph.microsoft.com/.default'
    }
    
    response = requests.post(url, headers=headers, data=data)
    return response.json().get("access_token")



def send_email_via_graph(recipient_email, attachment_path):
    access_token = get_access_token()
    
    # Get current date for the subject
    report_date = datetime.today().strftime("%d-%m-%Y")
    subject = f"RFM Automated Report ({report_date})"
    
    # Tech-Axion logo (Secure HTTPS URL)
    techaxion_logo_url = "https://www.techaxion.com/wp-content/uploads/2022/10/Axion-Logo.png"
    cc_recipients = ["ashish.xxxxx@xxxxxx.com", 'bhavik.xxxx@xxxxxxx.com']

    # """
    email_body = f"""
        <p>Dear Team,</p>
    
        <p>Kindly find the attached <b>RFM (Recency, Frequency, Monetary) automated reports</b> ({report_date}).</p>
    
        <p>This report categorizes clients based on their <b>trading behavior, brokerage contribution, and inactivity trends</b>. The goal is to identify high-value clients, potential growth opportunities, and clients at risk of inactivity.</p>
    
        <h3>🔹 Key Insights:</h3>
        <ul>
            <li><b>Highly engaged clients:</b> Frequent traders with significant brokerage contributions.</li>
            <li><b>Clients with declining activity:</b> May need re-engagement strategies.</li>
            <li><b>Inactive clients:</b> Require proactive outreach.</li>
        </ul>
    
        <h3>🔹 Client Segmentation:</h3>
        <ul>
            <li><b>Champions:</b> Most valuable clients, frequent traders.</li>
            <li><b>Loyal Customers:</b> Consistently active clients.</li>
            <li><b>Potential Loyalists:</b> Show promise but need nurturing.</li>
            <li><b>Promising:</b> Recently active, potential for growth.</li>
            <li><b>New Customers:</b> Recently onboarded clients.</li>
            <li><b>At Risk:</b> Clients with declining activity.</li>
            <li><b>Hibernating:</b> Inactive clients needing attention.</li>
        </ul>
    
        <p><b>Recommended Actions:</b> Based on their category, targeted strategies can be applied to retain, nurture, or re-engage clients.</p>
    
        <p>Best Regards,<br>
        <b>Ashish Ahirwar</b><br>
        BI Developer (Data Analyst)<br>
        Tech-Axion</p>
    
        <p><img src="https://www.xxxxx.com/wp-content/uploads/2022/10/Axion-Logo.png" alt="Tech-Axion Logo" width="200"></p>
    """

    # Read the attachment and encode it in Base64
    with open(attachment_path, "rb") as attachment_file:
        attachment_base64 = base64.b64encode(attachment_file.read()).decode('utf-8')

    # Email payload with attachment
    email_payload = {
        "message": {
            "subject": subject,
            "body": {
                "contentType": "HTML",
                "content": email_body
            },
            "toRecipients": [
                {"emailAddress": {"address": recipient_email}}
            ],
            "ccRecipients": [  # Adding CC recipients
                {"emailAddress": {"address": cc}} for cc in cc_recipients
            ],
            "attachments": [
                {
                    "@odata.type": "#microsoft.graph.fileAttachment",
                    "name": os.path.basename(attachment_path).split("/")[-1],  # Extracts the file name
                    "contentBytes": attachment_base64
                }
            ]
        }
    }

    # Send the email
    url = f"https://graph.microsoft.com/v1.0/users/{recipient_email}/sendMail"
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }

    response = requests.post(url, headers=headers, json=email_payload)

    # Check if the email was sent successfully
    if response.status_code == 202:
        print(f"Email sent successfully to {recipient_email}")
    else:
        print(f"Failed to send email to {recipient_email}: {response.text}")


# Connect to SQL Server and retrieve data
conn = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}')
query = """
            SELECT FY, CLIENT_ID, CLIENT_NAME, AGREEMENT_DATE, FIRST_TRADE, LAST_TRADE, FIRST_TRADE_DIFF, INACTIVE_SINCE, NO_OF_TRADES, TRADED_DAYS, 
            MKT_SESSIONS, GROSS_BRK, NET_BRK, TURNOVER, TOTAL_HOLDINGS, TOTAL_AUM, BRANCH_CODE, BRANCH_NAME, LEVEL_1, LEVEL_2, LEVEL_3, LEVEL_4,
            LEVEL_5, DIRECT_SUB_BROKER, INACTIVE_BUCKET, CATEGORY,DESCRIPTION, CHARACTERISTICS, ACTIONS, FILE_DATE, EXECUTION_TIME
            FROM RFM_MATRIX_PERC_SP 
            WHERE FY = 'FY 24-25' 
            """  # Replace 'your_table' with actual table name
df = pd.read_sql(query, conn)
conn.close()

# Ensure level columns exist
level_columns = ["LEVEL_1", "LEVEL_2", "LEVEL_3", "LEVEL_4"]
for col in level_columns:
    if col not in df.columns:
        print(f"Column {col} not found in data!")
        exit()

# Track processed persons to avoid duplicate reports
processed_persons = set()

# Iterate through level_1 to level_4
for level in level_columns:
    for person in df[level].dropna().unique():
        if person in processed_persons:
            continue  # Skip if already processed in a higher level
        
        # Filter data for this person
        person_data = df[df[level] == person]
        
        # Check if this person has an assigned email
        if person not in email_mapping:
            print(f"Skipping {person} (No email assigned)")
            continue
        
        recipient_email = email_mapping[person]
        
        # Generate Excel file
        timestamp = datetime.now().strftime("%Y%m%d")
        filename = f"{person.replace(' ', '_')}_Report_{timestamp}.xlsx"
        filepath = os.path.join(os.getcwd(), filename)
        
        # with pd.ExcelWriter(filepath, engine="openpyxl") as writer:
        #     person_data.to_excel(writer, index=False, sheet_name="Report")
            
       # Write DataFrame to Excel
        with pd.ExcelWriter(filepath, engine="openpyxl") as writer:
            person_data.to_excel(writer, index=False, sheet_name="Report")
            
        # Load workbook to apply formatting
        wb = load_workbook(filepath)
        ws = wb.active
        
        # Apply header styling (Background Color & Bold Text)
        header_fill = PatternFill(start_color="FFC000", end_color="FFC000", fill_type="solid")  # Gold color
        header_font = Font(bold=True, color="000000")  # Black text
        
        for cell in ws[1]:  # First row (header row)
            cell.fill = header_fill
            cell.font = header_font
        
        # Auto-adjust column width based on content
        for col in ws.columns:
            max_length = 0
            col_letter = col[0].column_letter  # Get column letter
            
            for cell in col:
                try:
                    max_length = max(max_length, len(str(cell.value)))
                except:
                    pass
            
            ws.column_dimensions[col_letter].width = max_length + 2  # Adding padding for better fit
        
        # Save the formatted workbook
        wb.save(filepath)
        
        print(f"Generated file: {filename}")
        
        # Send email
        email_subject = f"Report for {person}"
        email_body = f"Dear {person},\n\nPlease find your attached report.\n\nBest regards."
        send_email_via_graph(recipient_email, filepath)
        
        # Mark this person as processed
        processed_persons.add(person)

print("Process completed.")


C:\Users\ashish.ahirwar\AppData\Local\Temp\ipykernel_16984\3688891008.py:298: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Generated file: Praveen_Chaudhary_Report_20250227.xlsx
Email sent successfully to ashish.ahirwar@techaxion.com
Skipping HO Mgt (No email assigned)
Skipping Nirali Shah (No email assigned)
Skipping Himayat Ali (No email assigned)
Generated file: Amarendra_Sinha_Report_20250227.xlsx
Email sent successfully to ashish.ahirwar@techaxion.com
Skipping Vikas Pandya (No email assigned)
Skipping PCG Desk (No email assigned)
Skipping Mayur Soni (No email assigned)
Skipping Sub Broker (No email assigned)
Skipping Vijay Virkar (No email assigned)
Skipping Bhanu Prakash Mishra (No email assigned)
Skipping Sunil Deshpande (No email assigned)
Skipping Smita Mane (No email assigned)
Skipping Bhavesh Jobanputra (No email assigned)
Skipping Inactive (No email assigned)
Skipping Ravi Srivastava (No email assigned)
Skipping Trades App (No email assigned)
Skipping Nirali Shah (No email assigned)
Skipping Dinesh Modi (No email assigned)
Skipping Vinod P (No email assigned)
Skipping Dealing HO (No email assig

In [25]:
# For VAIBHAV & BHAVIK SIR 


import pandas as pd
import pyodbc
import requests
from datetime import datetime
import base64
import os
from openpyxl import Workbook
from openpyxl.styles import PatternFill, Font

# SQL Server connection details
server = 'xxx.xxx.xxx.xx'
database = 'xxxxxxx'
username = 'xxxxxx'
password = 'xxxxx'


client_id = 'xxxxxxxx-xx-xxx-xxxx-xxxxxxxxx'  
client_secret = 'xxxx~xxxxxxx-xxxxxxx'  
tenant_id = 'xxxxxx-xxxx-xxx-xxx-xxxxx' 


# Function to get access token from Microsoft Graph
def get_access_token():
    url = f'https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token'
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    data = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
        'scope': 'https://graph.microsoft.com/.default'
    }
    
    # Requesting the access token
    response = requests.post(url, headers=headers, data=data)
    
    if response.status_code == 200:
        return response.json().get('access_token')
    else:
        print("Failed to retrieve access token.")
        print(f"Response: {response.text}")  # Print response for debugging
        return None

# Function to fetch data and send email for each view
# Define the directory where you want to save the Excel files
save_directory = "C:\\Users\\ashish.xxxx\\Desktop\\API\\Excelbackup"

# Function to fetch data and send email for each view
# Function to fetch data and send email for each view or query
def process_view(view_or_query, report_name):
    # Connect to SQL Server and retrieve data
    try:
        conn = pyodbc.connect(
            f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'
        )
        
        # Check if the input is a query or a view name
        if view_or_query.strip().lower().startswith("select"):
            query = view_or_query  # Use as query if it starts with "SELECT"
        else:
            query = f"SELECT * FROM {view_or_query}"  # Otherwise, treat as a view name

        df = pd.read_sql(query, conn)
        conn.close()
        print(f"Data fetched successfully for {report_name}.")
    except Exception as e:
        print(f"Error in fetching data for {report_name}:", e)
        return

    # Ensure save directory exists
    os.makedirs(save_directory, exist_ok=True)
    
    # Export data to Excel with custom report name in the specified directory
    date_str = datetime.now().strftime("%d-%m-%Y")  # Change format to dd-mm-yyyy
    excel_filename = os.path.join(f"{report_name}.xlsx")

    # Create a new workbook and add a worksheet
    workbook = Workbook()
    worksheet = workbook.active
    worksheet.title = report_name

    # Write the DataFrame to the Excel worksheet including headers
    # Writing the headers first
    for col_num, column_title in enumerate(df.columns.values, 1):
        worksheet.cell(row=1, column=col_num, value=column_title)

    # Writing the data
    for r_idx, row in enumerate(df.itertuples(index=False), 2):  # Start from the second row
        for c_idx, value in enumerate(row, 1):
            worksheet.cell(row=r_idx, column=c_idx, value=value)

    # Formatting the header
    header_fill = PatternFill(start_color='0000FF', end_color='0000FF', fill_type='solid')  # Blue background
    header_font = Font(bold=True, color='FFFFFF')  # Bold white font

    for cell in worksheet[1]:  # First row is the header
        cell.fill = header_fill
        cell.font = header_font

    # Adjust column widths
    for column in worksheet.columns:
        max_length = 0
        column = [cell for cell in column]
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        adjusted_width = (max_length + 2)  # Adding some padding
        worksheet.column_dimensions[column[0].column_letter].width = adjusted_width

    # Save the workbook
    workbook.save(excel_filename)
    print(f"Data exported to Excel: {excel_filename}")

    return excel_filename  # Return the filename for later use


# List of views to process along with their custom report names
views_and_reports = [
            ("""SELECT ''''+[CLIENT_ID] AS CLIENT_ID,*
      
  FROM [Wealthone].[dbo].[RFM_MATRIX_PERC] 
  WHERE [FY] = 'FY 24-25'
    """, 'RFM Report FY 24-25')
        

]

# Store filenames for sending email later
filenames = []

# Process each view
for view, report in views_and_reports:
    filename = process_view(view, report)
    if filename:
        filenames.append(filename)  # Add the filename to the list

# Send email with all report attachments
access_token = get_access_token()
if access_token:
    email_api_url = 'https://graph.microsoft.com/v1.0/users/{userPrincipalName}/sendMail'
    date_str = datetime.now().strftime("%d-%m-%Y")  # Get the current date in dd-mm-yyyy format
    subject = f'RFM Automated Report Fy 24-25 {date_str}'  # Update subject to include date
    
    # Prepare the email body with your signature
    signature_image_path = "C:\\Users\\ashish.xxxx\\Downloads\\thumbnail_Outlook-oul3alwo.png"
    if os.path.exists(signature_image_path):
        with open(signature_image_path, "rb") as image_file:
            image_data = base64.b64encode(image_file.read()).decode('utf-8')

        body = (
                    f"<span style='font-family: Aptos; '>Kindly find the attached RFM(Recency, Frequency, Monetary) automated reports (<strong>{date_str}</strong>). Inside report you will find Clients are bifurcated by <strong>Recency</strong>, <strong>Frequency</strong> & <strong>Monetary</strong> model with (1,2,3) based ranking.<br><br>"
                    "<span style='font-family: Aptos; '>Column named <strong>Matrix:</strong><br>"
                    "<span style='font-family: Aptos; '>&emsp;- Matrix value = 111 represents the lowest value (least recent, least frequent, lowest monetary value).<br>"
                    "<span style='font-family: Aptos; '>&emsp;- Matrix value = 333 represents the highest value (most recent, most frequent, highest monetary value).<br>"
                    "<span style='font-family: Aptos; '>&emsp;- Matrix value = 113 represents the value (least recent, least frequent, higher monetary value). Similarly for other Matrix values.<br><br>"
                    "<span style='font-family: Aptos; '>Column named <strong>Category:</strong><br>"
                    "<span style='font-family: Aptos; '>&emsp;- Taking reference from Matrix values Clients are divided into this categories Champions, Loyal Customers, Potential Loyalists, Promising, New customers, At Risk, Can't Lose Them, Hibernating.<br><br>"
                    "<span style='font-family: Aptos; '>Column named <strong>Description, Characteristics, Actions:</strong><br>"
                    "<span style='font-family: Aptos; '>&emsp;- Focuses on giving more description, clarity of clients divided in different categories and Actions to be taken accordingly.<br><br>"
                    f"<span style='font-family: Constantia; '><light>Thanks & Regards,</light><br></span>"
                    f"<span style='font-family: Constantia;'><light>Ashish Ahirwar</light><br>"
                    "<light>BI Developer (Data Analyst)</light><br>"
                    "<light>Tech-Axion</light><br></span>"
                    f'<img src="data:image/png;base64,{image_data}" width="160" height="44"/>'  # Adjust the size as neededneeded
            )

    else:
        print("Signature image not found!")

     # Define the recipients
    to_recipients = [
        {"emailAddress": {"address": "ashish.ahirwar@xxxx.com"}},
        {"emailAddress": {"address": "ashish.ahirwar@xxxx.com"}}
    ]
    cc_recipients = [
        {"emailAddress": {"address": "ashish.ahirwar@xxxx.com"}},
        {"emailAddress": {"address": "ashish.ahirwar@xxxx.com"}}
    ]

    # Prepare the email message
    attachments = []
    for filename in filenames:
        with open(filename, "rb") as file:
            content_bytes = base64.b64encode(file.read()).decode('utf-8')
            attachments.append({
                "@odata.type": "#microsoft.graph.fileAttachment",
                "name": filename.split('/')[-1],  # Get the file name from the path
                "contentBytes": content_bytes
            })

    email_message = {
        "message": {
            "subject": subject,
            "body": {
                "contentType": "HTML",  # Change content type to HTML
                "content": body
            },
            "toRecipients": to_recipients,
            "ccRecipients": cc_recipients,
            "attachments": attachments  # Attach all files
        }
    }

    # Replace with the actual user's email address who has permissions
    response = requests.post(email_api_url.replace("{userPrincipalName}", "ashish.ahirwar@xxxx.com"),  
                             headers={'Authorization': f'Bearer {access_token}', 'Content-Type': 'application/json'}, 
                             json=email_message)
    
    if response.status_code == 202:
        print("Email sent successfully.")
    else:
        print("Error in sending email:", response.text)
else:
    print("Failed to retrieve access token.")


C:\Users\ashish.ahirwar\AppData\Local\Temp\ipykernel_16984\2003997477.py:61: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Data fetched successfully for RFM Report FY 24-25.
Data exported to Excel: RFM Report FY 24-25.xlsx
Email sent successfully.
